<a href="https://colab.research.google.com/github/pranavred/chatbot-ui/blob/main/PDF_Query_LLM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install openai
!pip install PyPDF2
!pip install faiss-cpu
!pip install tiktoken

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 530.5/530.5 kB 18.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.2/48.2 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 3.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 kB 9.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.3/70.3 kB 4.0 MB/s eta 0:00:00
  Attempting uninstall: SQLAlchemy
    Found existing installation: SQLAlchemy 2.0.9
    Uninstalli

In [ ]:
from PyPDF2 import PdfReader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import CharacterTextSplitter
from langchain.vectorstores import ElasticVectorSearch, Pinecone, Weaviate, FAISS

In [ ]:
# Get your API keys from openai, you will need to create an account. 
# Here is the link to get the keys: https://platform.openai.com/account/billing/overview
import os
os.environ["OPENAI_API_KEY"] = "OPENAI_AI_KEY"

In [ ]:
# connect your Google Drive
from google.colab import drive
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"

Mounted at /content/gdrive


In [23]:
# location of the pdf file/files. 
reader = PdfReader('/content/gdrive/My Drive/data/All my addresses and dates of visiting countries.pdf')

In [24]:
reader

In [25]:
# read data from the file and put them into a variable called raw_text
raw_text = ''
for i, page in enumerate(reader.pages):
    text = page.extract_text()
    if text:
        raw_text += text

In [26]:
# raw_text

In [27]:
raw_text[:100]

'A d d r e s s e s\nI\nl i v e d :\nF r o m\n2 0 2 1\nA p r\n0 1\nt o\n2 0 2 2\nJ u n\n3 0\nI\nl i v e d\na t\n9\nW '

In [28]:
# We need to split the text that we read into smaller chunks so that during information retreival we don't hit the token size limits. 

text_splitter = CharacterTextSplitter(        
    separator = "\n",
    chunk_size = 1000,
    chunk_overlap  = 200,
    length_function = len,
)
texts = text_splitter.split_text(raw_text)

In [29]:
len(texts)

4

In [30]:
texts[0]

'A d d r e s s e s\nI\nl i v e d :\nF r o m\n2 0 2 1\nA p r\n0 1\nt o\n2 0 2 2\nJ u n\n3 0\nI\nl i v e d\na t\n9\nW e s t\n3 1 s t\nS t ,\n1 8 B ,\nN e w\nY o r k ,\n1 0 0 0 1\nF r o m\n2 0 1 9\nF e b\n0 1\nt o\n2 0 2 1\nM a r\n3 1\nI\nl i v e d\na t\n4 0 1\nW e s t\n5 6 t h\nS t ,\n# 5 P ,\nN e w\nY o r k ,\nN Y\n1 0 0 1 9\nF r o m\n2 0 1 8\nO c t\n0 1\nt o\n2 0 1 9\nJ a n\n3 1\nI\nl i v e d\na t\n5 7 0\nW e s t\nE v e r g r e e n\nA v e\nB r o o k l y n ,\nN Y\n1 1 2 2 1\nF r o m\n2 0 1 7\nS e p\n0 1\nt o\n2 0 1 8\nS e p\n3 0\nI\nl i v e d\na t\n2 7 1\nW e s t\n4 7 t h ,\n1 8 A ,\nN e w\nY o r k ,\nN Y\n1 0 0 3 6\nF r o m\n2 0 1 6\nA u g\n0 1\nt o\n2 0 1 7\nA u g\n3 1\nI\nl i v e d\na t\n3 5 0\nW e s t\n4 3 r d\nS t ,\n2 1 B\nF r o m\n2 0 1 6\nJ a n\n0 1\nt o\n2 0 1 6\nJ u l y\n3 0\nI\nl i v e d\na t\n1 0 1\nJ U M P E R\nL N\nW E S T\nC H E S T E R ,\nP A\n1 9 3 8 2 - 6 1 6 0\nU n i t e d\nS t a t e s\nF r o m\n2 0 1 4\nA u g\n0 1\nt o\n2 0 1 5\nD e c\n3 1\nI\nl i v e d\na t\n2 7 5 2

In [31]:
texts[1]

"F r o m\n2 0 1 4\nA u g\n0 1\nt o\n2 0 1 5\nD e c\n3 1\nI\nl i v e d\na t\n2 7 5 2\nC a m p u s\nW a l k\nA v e ,\n2 4 F ,\nD u r h a m ,\nN C ,\n2 7 7 0 5F r om\nA pril\n2014\nt o\nJuly\n2014\n(Home\n-\nP ar ents\naddr ess)\nI\nliv ed\nat\nFlat\nNo.501,\n6-8-1030,\nNew\nBalaji\nResidency ,\nNGOs\nColony ,\nKT\nRoad,\nTirupati,\nAndhr a\nPr adesh\n-\n517507\nT h i s\nw a s\na l s o\nm y\np a r e n t ’ s\nh o m e .\nI\na l s o\nl i v e d\nh e r e\ni n\n2 0 1 2\nF r om\nMa y\n2013\nt o\nA pril\n2014\nI\nliv ed\nat\nFlat\nNo.506,\nSun\nV alle y\nApar tments,\nShilpa\nP ark,\nMaharshi\nSchool\nRoad,\nK ondapur ,\nHy der abad,\nAndhr a\nPr adesh\n-\n500084\nF r om\nA ugust\n2012\nt o\nMa y\n2013\nI\nliv ed\nat\nRoom\nNo.203,SVS\nMen ' s\nhostel,\nSai\nDutta\nComplex,\nNear\nRainbow\nHospital,Opp\nSilpar amam\nHitech\nCity ,\nOpposite\nSilpar amam,\nHy der abad,\nAndhr a\nPr adesh\n-\n500081\nC o l o m b i a\nV i s i t\n2 0 2 1\nI n d i a\nv i s i t\n2 0 2 2\nL e f t\nN Y C\nF e b\n1 2 ,\nA

In [32]:
# Download embeddings from OpenAI
embeddings = OpenAIEmbeddings()

In [33]:
docsearch = FAISS.from_texts(texts, embeddings)

In [34]:
docsearch

In [35]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI

In [36]:
chain = load_qa_chain(OpenAI(), chain_type="stuff")

In [40]:
query = "Where did I live in 2022?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' From 2021 Apr 01 to 2022 Jun 30 I lived at 9 West 31st St, 18B, New York, 10001.'

In [ ]:
query = "What was the cost of training the GPT4all model?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' $100'

In [ ]:
query = "How was the model trained?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The model was trained with LoRA (Hu et al., 2021) on the 437,605 post-processed examples for four epochs. Detailed model hyper-parameters and training code can be found in the associated repository and model training log.'

In [ ]:
query = "what was the size of the training dataset?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' The final training dataset contains 437,605 prompt-generation pairs.'

In [ ]:
query = "How is this different from other models?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

' This model is different from other models because it is based on LLaMA, it is licensed only for research purposes, and it is trained on a dataset of post-processed examples. It also has a TSNE visualization of the final training data, and a zoomed-in view to show generations related to personal health and wellness.'

In [ ]:
query = "What is Google Bard?"
docs = docsearch.similarity_search(query)
chain.run(input_documents=docs, question=query)

" I don't know."